Welcome, 

The aim of this project is try to make various classifications and identifications from a visual. Keras Conv2D Layers and 500 images will be use for each training models. You can get better results  by changing image sizes and model variations.  

**Introduction:**


1. Import necessary libraries 
2. Get info from dataset
3. Create list for each future (true or false)
4. Define function, reshape images to prepare training
5. Make models for each list
6. Train model
7. Make prediction 
8. Display result

Libraries

In [ ]:
import numpy as np
import pandas as pd
import os
import keras
import matplotlib.pyplot as plt
from cv2 import cv2 
from PIL import Image
from keras.utils import np_utils
from keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout


Get data info using pandas

In [ ]:
df = pd.read_csv('../input/celeba-dataset/list_attr_celeba.csv', encoding='utf-8')
df.head()

Create list for classify features. This list will use in models for prediction.

In [ ]:
male_list = df[(df['Male'] == 1)]['image_id'].head(500).values.tolist()
female_list = df[(df['Male'] == -1)]['image_id'].head(500).values.tolist()

glasses_list = df[(df['Eyeglasses'] == 1)]['image_id'].head(500).values.tolist()
no_glasses_list = df[(df['Eyeglasses'] == -1)]['image_id'].head(500).values.tolist()

smile_list = df[(df['Smiling'] == 1)]['image_id'].head(500).values.tolist()
notsmile_list = df[(df['Smiling'] == -1)]['image_id'].head(500).values.tolist()

makeup_list = df[(df['Heavy_Makeup'] == 1)]['image_id'].head(500).values.tolist()
nomakeup_list = df[(df['Heavy_Makeup'] == -1)]['image_id'].head(500).values.tolist()

bald_list = df[(df['Bald'] == 1)]['image_id'].head(500).values.tolist()
notbald_list = df[(df['Bald'] == -1)]['image_id'].head(500).values.tolist()

young_list = df[(df['Young'] == 1)]['image_id'].head(500).values.tolist()
notyoung_list = df[(df['Young'] == -1)]['image_id'].head(500).values.tolist()

wearhat_list = df[(df['Wearing_Hat'] == 1)]['image_id'].head(500).values.tolist()
notwearhat_list = df[(df['Wearing_Hat'] == -1)]['image_id'].head(500).values.tolist()

mustache_list = df[(df['Mustache'] == 1)]['image_id'].head(500).values.tolist()
nomustache_list = df[(df['Mustache'] == -1)]['image_id'].head(500).values.tolist()

goatee_list = df[(df['Goatee'] == 1)]['image_id'].head(500).values.tolist()
nogoatee_list = df[(df['Goatee'] == -1)]['image_id'].head(500).values.tolist()

chubby_list = df[(df['Chubby'] == 1)]['image_id'].head(500).values.tolist()
notchubby_list = df[(df['Chubby'] == -1)]['image_id'].head(500).values.tolist()

wavyhair_list = df[(df['Wavy_Hair'] == 1)]['image_id'].head(500).values.tolist()
nowavyhair_list = df[(df['Wavy_Hair'] == -1)]['image_id'].head(500).values.tolist()

attractive_list = df[(df['Attractive'] == 1)]['image_id'].head(500).values.tolist()
notattractive_list = df[(df['Attractive'] == -1)]['image_id'].head(500).values.tolist()

Defining some functions for multiprediction. Two type of variable here, yes or no. Such as male or female.. In function reading necessary images and converting to numpy arrays.

In [ ]:
img_width = 48
img_height = 48

def multimodels(model_name,x_list,y_list):
    data=[]
    img_pth = '../input/celeba-dataset/img_align_celeba/img_align_celeba/'
    labels=[]
    
    for val_x in x_list:
        imag=cv2.imread(img_pth+val_x)
        img_from_ar = Image.fromarray(imag, 'RGB')
        resized_image = img_from_ar.resize((img_width, img_height))
        data.append(np.array(resized_image))
        labels.append(0)
    for val_y in y_list:
        imag=cv2.imread(img_pth+val_y)
        img_from_ar = Image.fromarray(imag, 'RGB')
        resized_image = img_from_ar.resize((img_width, img_height))
        data.append(np.array(resized_image))
        labels.append(1)
        
        
    model_type=np.array(data)
    labels=np.array(labels)
        
    #optionally save np arrays for future use
    np.save(model_name+'.npy',model_type)
    np.save(model_name+'_labels.npy',labels)
    model_type=np.load(model_name+'.npy')
    labels=np.load(model_name+'_labels.npy')
        
    s=np.arange(model_type.shape[0])
    np.random.shuffle(s)
    model_type=model_type[s]
    labels=labels[s]
        
    num_classes=len(np.unique(labels))
    data_length=len(model_type)
        
    #Train test split
    (x_train,x_test)=model_type[(int)(0.1*data_length):],model_type[:(int)(0.1*data_length)]
    x_train = x_train.astype('float32')/255
    x_test = x_test.astype('float32')/255
        
    (y_train,y_test)=labels[(int)(0.1*data_length):],labels[:(int)(0.1*data_length)]
    y_train=keras.utils.to_categorical(y_train,num_classes)
    y_test=keras.utils.to_categorical(y_test,num_classes)
        
    # And Creating the model 
    #linear and sigmoid activation may output better result..
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(img_width, img_height,3)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    model.summary()
        
    #Fitting and compiling the model and adding the early stop to avoid overfitting
    model.compile(loss=keras.losses.binary_crossentropy, optimizer=keras.optimizers.Adamax(), metrics=['accuracy'])
    early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
    history = model.fit(x_train,y_train,batch_size=50, epochs=90,verbose=1, validation_split=0.33, callbacks=[early_stop])
    score =  model.evaluate(x_test, y_test, verbose=1)
        
    rslt = ('Test accuracy:', score[1])
    rslt
        
    #save model for future use to predict new images
    model.save(model_name+'_model.h5')

Definition is done in function, now lets train datas for models

In [ ]:
multimodels("gender",male_list,female_list)
multimodels("glasses", glasses_list, no_glasses_list)
multimodels("smiles", smile_list, notsmile_list)
multimodels("makeup", makeup_list, nomakeup_list)
multimodels("bald", bald_list, notbald_list)
multimodels("young", young_list, notyoung_list)
multimodels("wearhat", wearhat_list, notwearhat_list)
multimodels("mustache", mustache_list, nomustache_list)
multimodels("goatee", goatee_list, nogoatee_list)
multimodels("chubby", chubby_list, notchubby_list)
multimodels("wavyhair", wavyhair_list, nowavyhair_list)
multimodels("attractive", attractive_list, notattractive_list)

Models created and saved. Now import libraries for prediction side 

In [ ]:
from PIL import Image
import numpy as np
import os
import keras
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout
from cv2 import cv2
import webbrowser

Select image to predict

In [ ]:
img_dir = '../input/celeba-dataset/img_align_celeba/img_align_celeba/'
img_width = 48
img_height = 48
results = []
accuracies = []
model_img = "000001.jpg"

Define new function for prediction

In [ ]:
def predict_model(model_name, model_path, image, val_1, val_2):
    
    model = keras.models.load_model(model_path)
    model.summary()
    def convert_to_array(img):
        im = cv2.imread(img)
        img = Image.fromarray(im, 'RGB')
        image = img.resize((img_width,img_height))
        return np.array(image)
    def get_model_type_name(label):
        
        if label==0:
            results.append(val_1)
            return val_1
        if label==1:
            results.append(val_2)
            return val_2
    
    def predict_model_type(file):
        print("Predicting .................................")
        ar=convert_to_array(file)
        ar=ar/255
        a=[]
        a.append(ar)
        a=np.array(a)
        score=model.predict(a,verbose=1)
        #print(score)
        label_index=np.argmax(score)
        #print(label_index)
        acc=np.max(score)
        model_type=get_model_type_name(label_index)
        #print(model_type)
        print("The predicted model  "+model_type+" with accuracy =    "+str(acc))
        accuracies.append(str(acc))
    predict_model_type(image)

Predicting images using saved models

In [ ]:
im_path = img_dir+model_img
predict_model("gender","./gender_model.h5",img_dir+model_img,"is a male", "is a female")
predict_model("glasses","./glasses_model.h5",img_dir+model_img,"has glasses", "has no glasses")
predict_model("smiles","./smiles_model.h5",img_dir+model_img,"is smiling", "is not smiling")
predict_model("makeup","./makeup_model.h5",img_dir+model_img,"has heavy makeup", "has not heavy makeup")
predict_model("bald","./bald_model.h5",img_dir+model_img,"is bald", "is not bald")
predict_model("young","./young_model.h5",img_dir+model_img,"is young", "is not young")
predict_model("wearhat","./wearhat_model.h5",img_dir+model_img,"is wearing a hat", "is not wearing a hat")
predict_model("mustache","./mustache_model.h5",img_dir+model_img,"has mustache", "has not mustache")
predict_model("goatee","./goatee_model.h5",img_dir+model_img,"is goatee", "has not goatee")
predict_model("chubby","./chubby_model.h5",img_dir+model_img,"is chubby", "is not chubby")
predict_model("wavyhair","./wavyhair_model.h5",img_dir+model_img,"has wavyhair", "has not wavyhair")
predict_model("attractive","./attractive_model.h5",img_dir+model_img,"is attractive", "is not attractive")


Visualize output with html codes

In [ ]:
html = '''
    <div style="overflow: auto;  border: 2px solid #FFF8D8;
        padding: 2px; width: 300px;" >
        
        <div style="padding: 10px 0px 0px 10px; overflow: auto;">           
            <h4 style="margin-left: 50px; margin-top: 2px;">{}{}</h4>
            <h4 style="margin-left: 50px; margin-top: 2px;">{}{}</h4>
            <h4 style="margin-left: 50px; margin-top: 2px;">{}{}</h4>
            <h4 style="margin-left: 50px; margin-top: 2px;">{}{}</h4>
            <h4 style="margin-left: 50px; margin-top: 2px;">{}{}</h4>
            <h4 style="margin-left: 50px; margin-top: 2px;">{}{}</h4>
            <h4 style="margin-left: 50px; margin-top: 2px;">{}{}</h4>
            <h4 style="margin-left: 50px; margin-top: 2px;">{}{}</h4>
            <h4 style="margin-left: 50px; margin-top: 2px;">{}{}</h4>
            <h4 style="margin-left: 50px; margin-top: 2px;">{}{}</h4>
            <h4 style="margin-left: 50px; margin-top: 2px;">{}{}</h4>
	    <h4 style="margin-left: 50px; margin-top: 2px;">{}{}</h4>
            
        </div>
    </div>
    '''.format("gender= "+results[0],", acc: "+accuracies[0][:4],
                "glasses= "+results[1],",  acc : "+accuracies[1][:4],
                "smiling= "+results[2], ",  acc : "+accuracies[2][:4], 
                "makeup= "+results[3], ",  acc: "+accuracies[3][:4],
                "bald= "+results[4], ",  acc: "+accuracies[4][:4],
                "young= "+results[5], ",  acc: "+accuracies[5][:4],
                "wearhat= "+results[6], ",  acc: "+accuracies[6][:4],
                "mustache= "+results[7], ",  acc: "+accuracies[7][:4],
                "goatee= "+results[8], ",  acc: "+accuracies[8][:4],
                "chubby= "+results[9], ",  acc: "+accuracies[9][:4],
	        "wavyhair= "+results[10], ",  acc: "+accuracies[10][:4],
                "attractive= "+results[11], ",  acc: "+accuracies[11][:4] )

Display image and prediction..

In [ ]:
from IPython.display import HTML, Image
import IPython.display
im_path = img_dir+model_img
display(HTML(html))
Image(filename=im_path)

Thanks